In [ ]:
import numpy as np
import torch
import os
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import torchvision.models as models
from PIL import Image
import cv2
from torchvision.utils import save_image
from true_classify import *
from Utils import *
from anonymization_methods import *
from datasets import *
from torchvision.transforms.functional import to_pil_image
from collections import Counter
import matplotlib.image as mpimg
import time
import matplotlib.pyplot as plt
import xlrd
import openpyxl

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# define training and testing data directories
source_path = 'D:/Younas_Work/D2_Final/Original/All FR/train'

class_names = [folder for folder in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, folder))]
num_classes = len(class_names)

file_list = os.listdir(source_path)
model_dir = 'Convnext_pretrained_younas.pt'

output_path = 'D:/Younas_Work/D2_Final/Anonymized/Expite/Simple_Pix_Old_15/Face_Recognition'

save_roc_dir = 'D:/Younas_Work/D2_Final/Results_Final/ROC/ROC Plots Mouth_Masked/'
excel_file_path = 'D:/Younas_Work/D2_Final/Results_Final/Excel_Sheets/Mouth_Masked.xlsx'

In [ ]:

# Define a new transform with additional data augmentations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
model = models.convnext_base(pretrained=True)
model.classifier[2]=nn.Linear(1024,num_classes)

model.load_state_dict(torch.load('Convnext_pretrained_younas.pt'))
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.001)

In [ ]:
# We will try a fixed q in each iteration
q=90

In [ ]:
#Setting Parameter values

# we will put all the values to highest possible to minimize the changes.

#For Simple Blur
blur_kernel_size = 3

#For DP Blur
#block
b = 16
#m & eps are for privacy budget. Larger m = more noise
m = 2
#eps is the privacy parameter in DP, smaller eps means higher noise and more distortion
eps = 100
#K is kernel size for gaussian blur. Height and width of the Gaussian kernel
k = 9
#SD of Gaussian blur. Larger sigma means more blurring
sigma = 0

#For Simple Pixelate
pixel_size = 3


#For DP Pixelate
#b, m and eps are the same for DP pixelate and DP Blur

In [ ]:
# new_batch_size = 1
# our_test_loader = create_test_loader(source_path, new_batch_size)
# final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
# prev_acc = final_acc
# prev_acc*100

In [ ]:
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr, roc_auc, dataset_name, save_dir):
    plt.figure()
    lw = 2

    # Plot macro-average ROC curve
    plt.plot(fpr['micro'], tpr['micro'], color='deeppink', linestyle=':', lw=lw,
             label='Macro-average ROC curve (area = {0:0.2f})'.format(roc_auc['micro']))

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC for ' + dataset_name)
    plt.legend(loc="lower right")

    plt.savefig(os.path.join(save_dir, f'ROC_{dataset_name}.png'))
    plt.close()

In [ ]:
import torch
from tqdm import tqdm
import numpy as np
from sklearn.metrics import roc_curve, f1_score, auc, precision_score, recall_score
import pandas as pd
import os
import xlrd
import openpyxl


def test_images_classification(original_model, device, local_test_loader, excel_file_path, save_roc_dir):
    original_model.eval()
    # Accuracy counter
    correct = 0
    correct_examples = []
    logits = []
    labels = []
    counter = 0
    
    excel_dir = os.path.dirname(excel_file_path)

    # Extract the dataset name from the complete path of local_test_loader
    dataset_name = os.path.basename(local_test_loader.dataset.root)

    # Extract the complete path of local_test_loader
    complete_path = local_test_loader.dataset.root

    # Loop over all examples in the test set
    for data, target in tqdm(local_test_loader):
        counter += 1

        # Send the data and label to the device
        data = data.to(device)
        target = target.to(device)

        # Forward pass the data through the model
        with torch.no_grad():
            output = original_model(data)

        final_pred = torch.argmax(output, dim=1)
        if final_pred.item() == target.item():
            correct += 1
        correct_examples.append(data)
        labels.append(target)
        logits.append(output)
    final_acc = correct / float(len(local_test_loader))

    # Calculate the F1 score, precision, and recall
    all_logits = torch.cat(logits, dim=0)
    all_labels = torch.cat(labels, dim=0)
    predicted_labels = torch.argmax(all_logits, dim=1)
    f1 = f1_score(all_labels.cpu(), predicted_labels.cpu(), average='macro')
    precision = precision_score(all_labels.cpu(), predicted_labels.cpu(), average='macro')
    recall = recall_score(all_labels.cpu(), predicted_labels.cpu(), average='macro')

    # Calculate and save the ROC curve data
    all_probs = torch.nn.functional.softmax(all_logits, dim=1)
    num_classes = all_probs.shape[1]
    one_hot_labels = torch.zeros_like(all_probs).scatter_(1, all_labels.view(-1, 1), 1)

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(one_hot_labels[:, i].cpu(), all_probs[:, i].cpu())
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Micro-average ROC curve and AUC
    fpr["micro"], tpr["micro"], _ = roc_curve(one_hot_labels.cpu().ravel(), all_probs.cpu().ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Save summary to Excel
    if os.path.exists(excel_file_path):
        df_summary = pd.read_excel(excel_file_path, sheet_name="Summary", engine="openpyxl")
        new_row = {"Complete Path": complete_path, "Accuracy": final_acc, "F1 Score": f1, "Precision": precision, "Recall": recall, "AUC": roc_auc["micro"]}
        df_summary = df_summary.append(new_row, ignore_index=True)
        df_summary.to_excel(excel_file_path, sheet_name="Summary", index=False)
    else:
        summary_data = {"Complete Path": [complete_path], "Accuracy": [final_acc], "F1 Score": [f1], "Precision": [precision], "Recall": [recall], "AUC": [roc_auc["micro"]]}
        df_summary = pd.DataFrame(summary_data)
        df_summary.to_excel(excel_file_path, sheet_name="Summary", index=False)
    
    # Plot and save the ROC curve
    plot_roc_curve(fpr, tpr, roc_auc, dataset_name, save_roc_dir)

    # Return the accuracy, F1 score, and an adversarial example
    return final_acc, correct_examples, labels, logits

In [ ]:
start_time = time.time()
new_batch_size = 1
iteration_out_path = 'D:/Younas_Work/SOTA/DP/'
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
prev_acc = final_acc
print(prev_acc*100)
end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time : {overall_execution_time} seconds")

In [ ]:
Train

100

961


91.44109797941289

Overall execution time : 18709.195513010025 seconds
    

test
72.36421725239617

Overall execution time : 879.967887878418 seconds
    
68.97590361445783

Overall execution time : 236.98045086860657 seconds
    
    
Val
    
68.97590361445783

Overall execution time : 236.98045086860657 seconds

In [ ]:
24.05789186237029

Overall execution time : 4703.626936912537 seconds

In [ ]:
# 'D:/Younas_Work/D2_Final/Anonymized/Original/Simple_Pix/BS_25/Face_Recognition'
# Acc: 24.05789186237029
# Time: 2209.5845515727997 seconds
    
# 'D:/Younas_Work/D2_Final/Anonymized/Original/Simple_Pix/BS_50/Face_Recognition'
# Acc: 2.3211359912616056
# Time: 3743.299655675888 seconds
    
# 'D:/Younas_Work/D2_Final/Anonymized/Original/Simple_Pix/BS_75/Face_Recognition'
# Acc:
# Time: 

# 'D:/Younas_Work/D2_Final/Anonymized/Original/Simple_Pix/BS_100/Face_Recognition'
# Acc:
# Time:

In [ ]:
start_time = time.time()
new_batch_size = 1
iteration_out_path = 'D:/Younas_Work/D2_Final/Anonymized/Expite/Simple_Pix_Old_15/Face_Recognition/Iteration_15'
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
prev_acc = final_acc
print(prev_acc*100)

end_time = time.time()
acc_time = end_time - start_time
print(f"\nAcc execution time: {acc_time} seconds")

start_time = time.time()

for itera in range(16,17):
    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)

        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"

        class_label = labels[i]
        class_subfolder = class_names[class_label]            

        # Create subfolder based on the class name
    #         class_subfolder = class_names[correct_label.item()]

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True

        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)


        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

    # Call different perturbation functions here
#             perturbed = create_blurred_image(x, blur_kernel_size)
            perturbed = create_pixelated_image(x, pixel_size)
#             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#            perturbed = create_dp_pixelated_image(x, b, m, eps)


            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients

            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)

end_time = time.time()
Anon_execution_time = end_time - start_time
print(f"\nAnon Time: {Anon_execution_time} seconds")
print(f"All images processed for iteration number {itera}.")

In [ ]:
start_time = time.time()
patience = 5

for itera in range(1,2):

    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)

        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"

        class_label = labels[i]
        class_subfolder = class_names[class_label]            

        # Create subfolder based on the class name
    #         class_subfolder = class_names[correct_label.item()]

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True

        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)


        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

    # Call different perturbation functions here
#             perturbed = create_blurred_image(x, blur_kernel_size)
    #             perturbed = create_pixelated_image(x, pixel_size)
    #             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
            perturbed = create_dp_pixelated_image(x, b, m, eps)


            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients

            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)
    print(f"All images processed for iteration number {itera}.")
    
    new_test_path = iteration_out_path
    our_test_loader = create_test_loader(new_test_path, new_batch_size)
    # get the list of correctly classified images
    iter_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
    print(f"The accuracy after iteration {itera} is {iter_acc*100}%")
    
    if (iter_acc > prev_acc):
        patience -= 1
        prev_acc = iter_acc
        print("No Drop")
        if patience == 0:
            break
    else:
        prev_acc = iter_acc
        patience = 5
        
    if (iter_acc < .01):
        break

end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")
print("All images processed.")

In [ ]:
# Loop until all quantile values have been processed
start_time = time.time()
new_batch_size = 1
iteration_out_path = f"{output_path}/Iteration_4/"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, "EDI_Simple_Blur.xlsx")
prev_acc = final_acc
print(prev_acc*100)

end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")

start_time2 = time.time()


patience = 5
for itera in range(5,10):
    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"
        
        class_label = labels[i]
        class_subfolder = class_names[class_label]            
        
        # Create subfolder based on the class name
#         class_subfolder = class_names[correct_label.item()]

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data
            
            
            # Call different perturbation functions here
            perturbed = create_blurred_image(x, blur_kernel_size)
#            perturbed = create_pixelated_image(x, pixel_size)
#            perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#            perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
    
            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)
        
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    
    our_test_loader = create_test_loader(new_test_path, new_batch_size)
    # get the list of correctly classified images
    iter_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, "EDI_Simple_Blur.xlsx")
    print(f"The accuracy after iteration {itera} is {iter_acc*100}%")
    
    if (iter_acc > prev_acc):
        patience -= 1
        prev_acc = iter_acc
        print("No Drop")
        if patience == 0:
            break
    else:
        prev_acc = iter_acc
        patience = 5
        
    if (iter_acc < .01):
        break

end_time2 = time.time()
overall_execution_time2 = end_time2 - start_time2
print(f"\nOverall execution time: {overall_execution_time2} seconds")
print("All images processed.")

In [ ]:
# Loop until all quantile values have been processed
start_time = time.time()
new_batch_size = 1
iteration_out_path = f"{output_path}/Iteration_9/"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
prev_acc = final_acc
print(prev_acc*100)


patience = 5
for itera in range(10,15):
    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"
        
        class_label = labels[i]
        class_subfolder = class_names[class_label]            
        
        # Create subfolder based on the class name
#         class_subfolder = class_names[correct_label.item()]

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data
            
            # Call different perturbation functions here
#             perturbed = create_pixelated_image(x, pixel_size)

#            perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
            perturbed = create_blurred_image(x, blur_kernel_size)
    
            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)
        
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    
    our_test_loader = create_test_loader(new_test_path, new_batch_size)
    # get the list of correctly classified images
    iter_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
    print(f"The accuracy after iteration {itera} is {iter_acc*100}%")
    
    if (iter_acc > prev_acc):
        patience -= 1
        prev_acc = iter_acc
        print("No Drop")
        if patience == 0:
            break
    else:
        prev_acc = iter_acc
        patience = 5
        
    if (iter_acc < .01):
        break

end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")
print("All images processed.")

In [ ]:
# Loop until all quantile values have been processed
start_time = time.time()
new_batch_size = 1
iteration_out_path = f"{output_path}/Iteration_14/"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
prev_acc = final_acc
print(prev_acc*100)


patience = 5
for itera in range(15,20):
    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"
        
        class_label = labels[i]
        class_subfolder = class_names[class_label]            
        
        # Create subfolder based on the class name
#         class_subfolder = class_names[correct_label.item()]

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data
            
            # Call different perturbation functions here
            perturbed = create_pixelated_image(x, pixel_size)

#            perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#             perturbed = create_blurred_image(x, blur_kernel_size)
    
            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)
        
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    
    our_test_loader = create_test_loader(new_test_path, new_batch_size)
    # get the list of correctly classified images
    iter_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
    print(f"The accuracy after iteration {itera} is {iter_acc*100}%")
    
    if (iter_acc > prev_acc):
        patience -= 1
        prev_acc = iter_acc
        print("No Drop")
        if patience == 0:
            break
    else:
        prev_acc = iter_acc
        patience = 5
        
    if (iter_acc < .01):
        break

end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")
print("All images processed.")

In [ ]:
# Loop until all quantile values have been processed
start_time = time.time()
new_batch_size = 1
iteration_out_path = f"{output_path}/Iteration_14/"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
prev_acc = final_acc
print(prev_acc*100)


patience = 5
for itera in range(15,20):
    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"
        
        class_label = labels[i]
        class_subfolder = class_names[class_label]            
        
        # Create subfolder based on the class name
#         class_subfolder = class_names[correct_label.item()]

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

            # Call different perturbation functions here

#             perturbed = create_dp_pixelated_image(x, b, m, eps)
            perturbed = create_pixelated_image(x, pixel_size)
#             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#             perturbed = create_blurred_image(x, blur_kernel_size)
    
            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)
        
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    
    our_test_loader = create_test_loader(new_test_path, new_batch_size)
    # get the list of correctly classified images
    iter_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
    print(f"The accuracy after iteration {itera} is {iter_acc*100}%")
    
    if (iter_acc > prev_acc):
        patience -= 1
        prev_acc = iter_acc
        print("No Drop")
        if patience == 0:
            break
    else:
        prev_acc = iter_acc
        patience = 5
        
    if (iter_acc < .01):
        break

end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")
print("All images processed.")

In [ ]:
start_time = time.time()
new_batch_size = 1
iteration_out_path = f"{output_path}/Iteration_18/"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
prev_acc = final_acc


# Loop until all quantile values have been processed
patience = 5
for itera in range(19,20):

    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"
        # Create subfolder based on the class name
        class_subfolder = class_names[correct_label.item()]
        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        
        
        if not os.path.exists(class_output_path):
            os.makedirs(class_output_path)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

            # Call different perturbation functions here

            perturbed = create_dp_pixelated_image(x, b, m, eps)
#             perturbed = create_pixelated_image(x, pixel_size)
#             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#             perturbed = create_blurred_image(x, blur_kernel_size)
    
            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)
        
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    
    our_test_loader = create_test_loader(new_test_path, new_batch_size)
    # get the list of correctly classified images
    iter_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
    print(f"The accuracy after iteration {itera} is {iter_acc*100}%")
    
    if (iter_acc > prev_acc):
        patience -= 1
        prev_acc = iter_acc
        print("No Drop")
        if patience == 0:
            break
    else:
        prev_acc = iter_acc
        patience = 5
end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")

print("All images processed.")

In [ ]:
new_batch_size = 1
iteration_out_path = f"{output_path}/iteration_9"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)

In [ ]:
final_acc*100

In [ ]:
# Loop until all quantile values have been processed
for itera in range(10,15):

    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/iteration_{itera}/"
        # Create subfolder based on the class name
        class_subfolder = class_names[correct_label.item()]
        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        
        if not os.path.exists(class_output_path):
            os.makedirs(class_output_path)

        
        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

            # Call different perturbation functions here

#             perturbed = create_pixelated_image(x, pixel_size)
#             perturbed = create_pixelated_image(x, pixel_size)
            perturbed = create_dp_pixelated_image(x, b, m, eps)
#             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#             perturbed = create_blurred_image(x, blur_kernel_size)
    
            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
#             annonymized_image.requires_grad = False
            annonymized_image = annonymized_image - updated_noise
        

    #         # Calculate the output and loss for the anonymized and original images
    #         output, _ = calculate_loss(model, annonymized_image, y, criterion)
    #         output_original, _ = calculate_loss(model, x.clone(), y, criterion)

    #         # Determine the new predicted label for both the anonymized and original images
    #         new_label = torch.argmax(output, dim=1)
    #         output_original = torch.argmax(output_original, dim=1)

            # Save the anonymized image to the output directory
            save_image_ours(annonymized_image, i, correct_label, class_output_path, eps, q)
        
# #         save_image(annonymized_image, i, correct_label, iteration_out_path, eps, q)    
        else:
#             annonymized_image.requires_grad = False
            save_image_ours(annonymized_image, i, correct_label, class_output_path, eps, q)
        
#         print(f"Image {i} processed for q={q}.")
    
    # Decrement q for the next iteration
#     q -= 3
#     iteration_number +=1
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    our_test_loader = create_test_loader(new_test_path, new_batch_size)
#     _,_,our_test_loader = create_data_loader(train_path, new_test_path, new_batch_size)
    # get the list of correctly classified images
    final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
    print(f"The accuracy after the iteration {itera} is {final_acc*100}%")
    
print("All images processed.")

In [ ]:
new_batch_size = 1
iteration_out_path = f"{output_path}/iteration_14/"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)

In [ ]:
final_acc*100

In [ ]:
# Loop until all quantile values have been processed
for itera in range(15,20):

    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/iteration_{itera}/"
        # Create subfolder based on the class name
        class_subfolder = class_names[correct_label.item()]
        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        
        if not os.path.exists(class_output_path):
            os.makedirs(class_output_path)

        
        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

            # Call different perturbation functions here

#             perturbed = create_blurred_image(x, blur_kernel_size)
            perturbed = create_pixelated_image(x, pixel_size)

#             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)

            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
#             annonymized_image.requires_grad = False
            annonymized_image = annonymized_image - updated_noise
        

    #         # Calculate the output and loss for the anonymized and original images
    #         output, _ = calculate_loss(model, annonymized_image, y, criterion)
    #         output_original, _ = calculate_loss(model, x.clone(), y, criterion)

    #         # Determine the new predicted label for both the anonymized and original images
    #         new_label = torch.argmax(output, dim=1)
    #         output_original = torch.argmax(output_original, dim=1)

            # Save the anonymized image to the output directory
            save_image_ours(annonymized_image, i, correct_label, class_output_path, eps, q)
        
# #         save_image(annonymized_image, i, correct_label, iteration_out_path, eps, q)    
        else:
#             annonymized_image.requires_grad = False
            save_image_ours(annonymized_image, i, correct_label, class_output_path, eps, q)
        
#         print(f"Image {i} processed for q={q}.")
    
    # Decrement q for the next iteration
#     q -= 3
#     iteration_number +=1
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    our_test_loader = create_test_loader(new_test_path, new_batch_size)
#     _,_,our_test_loader = create_data_loader(train_path, new_test_path, new_batch_size)
    # get the list of correctly classified images
    final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
    print(f"The accuracy after the iteration {itera} is {final_acc*100}%")
    
print("All images processed.")

In [ ]:
new_batch_size = 1
iteration_out_path = f"{output_path}/iteration_12/"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)

In [ ]:
final_acc

In [ ]:
# Loop until all quantile values have been processed
for itera in range(13,16):

    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/iteration_{itera}/"
        # Create subfolder based on the class name
        class_subfolder = class_names[correct_label.item()]
        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        
        if not os.path.exists(class_output_path):
            os.makedirs(class_output_path)

        
        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

            # Call different perturbation functions here

#             perturbed = create_blurred_image(x, blur_kernel_size)
#             perturbed = create_pixelated_image(x, pixel_size)

            perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)

            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
#             annonymized_image.requires_grad = False
            annonymized_image = annonymized_image - updated_noise
        

    #         # Calculate the output and loss for the anonymized and original images
    #         output, _ = calculate_loss(model, annonymized_image, y, criterion)
    #         output_original, _ = calculate_loss(model, x.clone(), y, criterion)

    #         # Determine the new predicted label for both the anonymized and original images
    #         new_label = torch.argmax(output, dim=1)
    #         output_original = torch.argmax(output_original, dim=1)

            # Save the anonymized image to the output directory
            save_image_ours(annonymized_image, i, correct_label, class_output_path, eps, q)
        
# #         save_image(annonymized_image, i, correct_label, iteration_out_path, eps, q)    
        else:
#             annonymized_image.requires_grad = False
            save_image_ours(annonymized_image, i, correct_label, class_output_path, eps, q)
        
#         print(f"Image {i} processed for q={q}.")
    
    # Decrement q for the next iteration
#     q -= 3
#     iteration_number +=1
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    _,_,our_test_loader = create_data_loader(train_path, new_test_path, new_batch_size)
    # get the list of correctly classified images
    final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
    print(f"The accuracy after the iteration {itera} is {final_acc*100}%")
    
print("All images processed.")

In [ ]:
new_batch_size = 1
iteration_out_path = f"{output_path}/iteration_15/"
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)

In [ ]:
final_acc

In [ ]:
# Loop until all quantile values have been processed
for itera in range(16,21):

    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)
        
        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/iteration_{itera}/"
        # Create subfolder based on the class name
        class_subfolder = class_names[correct_label.item()]
        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        
        if not os.path.exists(class_output_path):
            os.makedirs(class_output_path)

        
        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True
        
        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)
        
        
        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

            # Call different perturbation functions here

#             perturbed = create_blurred_image(x, blur_kernel_size)
#             perturbed = create_pixelated_image(x, pixel_size)

            perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)

            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients
            
            
            
            # Create the anonymized image by subtracting/adding the updated noise from the original image
#             annonymized_image.requires_grad = False
            annonymized_image = annonymized_image - updated_noise
        

    #         # Calculate the output and loss for the anonymized and original images
    #         output, _ = calculate_loss(model, annonymized_image, y, criterion)
    #         output_original, _ = calculate_loss(model, x.clone(), y, criterion)

    #         # Determine the new predicted label for both the anonymized and original images
    #         new_label = torch.argmax(output, dim=1)
    #         output_original = torch.argmax(output_original, dim=1)

            # Save the anonymized image to the output directory
            save_image_ours(annonymized_image, i, correct_label, class_output_path, eps, q)
        
# #         save_image(annonymized_image, i, correct_label, iteration_out_path, eps, q)    
        else:
#             annonymized_image.requires_grad = False
            save_image_ours(annonymized_image, i, correct_label, class_output_path, eps, q)
        
#         print(f"Image {i} processed for q={q}.")
    
    # Decrement q for the next iteration
#     q -= 3
#     iteration_number +=1
    print(f"All images processed for iteration number {itera}.")
    new_test_path = iteration_out_path
    _,_,our_test_loader = create_data_loader(train_path, new_test_path, new_batch_size)
    # get the list of correctly classified images
    final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader)
    print(f"The accuracy after the iteration {itera} is {final_acc*100}%")
    
print("All images processed.")